<a href="https://colab.research.google.com/github/cs-pub-ro/ML/blob/master/lab/lab6/Laborator_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Învățare Automată
# Învățare prin Recompensă - rezolvarea proceselor de decizie Markov prin tehnici de programare dinamică (Value Iteration, Policy Iteration)
### Autori:
* Tudor Berariu - 2018
* Alexandru Sorici - 2020

## 1. Scopul laboratorului

Scopul laboratorului îl reprezintă înțelegerea conceptelor de proces markov de decizie (MDP), politică, valoare de stare, precum și implementarea unor metode de programare dinamică pentru rezolvarea problemei de control a unui MDP.

În cadrul laboratorului veți:
- implementa algoritmul de iterare a politicilor
- implementa algoritmul de iterare a valorilor de stare

## 2. Workspace setup

### Câteva bibioteci de care vom avea nevoie

In [ ]:
import sys
import os.path
from argparse import ArgumentParser
from copy import copy
from random import choice

from typing import Dict, List, Tuple

### Definirea unui labirint

In [ ]:
class Maze:

    NORTH, EAST, SOUTH, WEST = 0, 1, 2, 3  # actions

    DYNAMICS = {  # the stochastic effects of actions
        NORTH: {(0, -1): 0.1, (-1, 0): .8, (0, 1): .1},
        EAST: {(-1, 0): 0.1, (0, 1): .8, (1, 0): .1},
        SOUTH: {(0, 1): 0.1, (1, 0): .8, (0, -1): .1},
        WEST: {(1, 0): 0.1, (0, -1): .8, (-1, 0): .1},
    }

    WALL, EMPTY = "x", " "

    VISUALS = {
        (0, 0, 1, 1): "\N{BOX DRAWINGS HEAVY DOWN AND RIGHT}",
        (1, 0, 0, 1): "\N{BOX DRAWINGS HEAVY DOWN AND LEFT}",
        (1, 0, 1, 0): "\N{BOX DRAWINGS HEAVY HORIZONTAL}",
        (0, 1, 1, 0): "\N{BOX DRAWINGS HEAVY UP AND RIGHT}",
        (1, 1, 0, 0): "\N{BOX DRAWINGS HEAVY UP AND LEFT}",
        (0, 1, 0, 1): "\N{BOX DRAWINGS HEAVY VERTICAL}",
        (1, 1, 1, 1): "\N{BOX DRAWINGS HEAVY VERTICAL AND HORIZONTAL}",
        (1, 1, 1, 0): "\N{BOX DRAWINGS HEAVY UP AND HORIZONTAL}",
        (1, 1, 0, 1): "\N{BOX DRAWINGS HEAVY VERTICAL AND LEFT}",
        (1, 0, 1, 1): "\N{BOX DRAWINGS HEAVY DOWN AND HORIZONTAL}",
        (0, 1, 1, 1): "\N{BOX DRAWINGS HEAVY VERTICAL AND RIGHT}",
        (1, 0, 0, 0): "\N{BOX DRAWINGS HEAVY LEFT}",
        (0, 1, 0, 0): "\N{BOX DRAWINGS HEAVY UP}",
        (0, 0, 1, 0): "\N{BOX DRAWINGS HEAVY RIGHT}",
        (0, 0, 0, 1): "\N{BOX DRAWINGS HEAVY DOWN}",
        (0, 0, 0, 0): "\N{BOX DRAWINGS HEAVY VERTICAL AND HORIZONTAL}",
        WEST: "\N{LEFTWARDS ARROW}",
        NORTH: "\N{UPWARDS ARROW}",
        EAST: "\N{RIGHTWARDS ARROW}",
        SOUTH: "\N{DOWNWARDS ARROW}",
    }

    def __init__(self, map_name):
        self._rewards, self._cells = {}, []
        with open(os.path.join("maps", map_name), "r") as map_file:
            for line in map_file.readlines():
                if ":" in line:
                    name, value = line.strip().split(":")
                    self._rewards[name] = float(value)
                else:
                    self._cells.append(list(line.strip()))
        self._states = [(i, j) for i, row in enumerate(self._cells)
                        for j, cell in enumerate(row) if cell != Maze.WALL]

    @property
    def actions(self):
        return [Maze.NORTH, Maze.EAST, Maze.SOUTH, Maze.WEST]

    @property
    def states(self):
        return copy(self._states)

    def is_final(self, state):
        row, col = state
        return self._cells[row][col] != Maze.EMPTY

    def effects(self, state, action):
        if self.is_final(state):
            return []
        row, col = state
        next_states = {}
        for (d_row, d_col), prob in Maze.DYNAMICS[action].items():
            next_row, next_col = row + d_row, col + d_col
            if self._cells[next_row][next_col] == Maze.WALL:
                next_row, next_col = row, col
            if (next_row, next_col) in next_states:
                prev_prob, _ = next_states[(next_row, next_col)]
                prob += prev_prob
            cell = self._cells[next_row][next_col]
            reward = self._rewards["default" if cell == Maze.EMPTY else cell]
            next_states[(next_row, next_col)] = (prob, reward)
        return [(s, p, r) for s, (p, r) in next_states.items()]

    def print_policy(self, policy):
        last_row = []
        height = len(self._cells)

        for row, row_cells in enumerate(self._cells):
            width = len(row_cells)
            for col, cell in enumerate(row_cells):
                if cell == Maze.WALL:
                    north, south, west, east = 0, 0, 0, 0
                    if last_row and len(last_row) > col:
                        north = last_row[col] == Maze.WALL
                    if row + 1 < height:
                        south = self._cells[row + 1][col] == Maze.WALL
                    if col > 0:
                        west = row_cells[col - 1] == Maze.WALL
                    if col + 1 < width:
                        east = row_cells[col + 1] == Maze.WALL
                    sys.stdout.write(Maze.VISUALS[(west, north, east, south)])
                elif self.is_final((row, col)):
                    sys.stdout.write(cell)
                else:
                    action = policy[(row, col)]
                    sys.stdout.write(Maze.VISUALS[action])
            sys.stdout.write("\n")
            last_row = row_cells
        sys.stdout.flush()

    def print_values(self, v):
        for r, row_cells in enumerate(self._cells):
            print(" | ".join(["%5.2f" % v[(r, c)] if cell == Maze.EMPTY else "     "
                              for c, cell in enumerate(row_cells)]))

## Cerințe

In [ ]:
MAP_NAME = 'simple'  #@param ['simple', 'complex', 'be_careful', 'suffer']
gamma = 0.8 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
max_delta = 1e-8 #@param {type:"float"}.

### Cerința 1
Implementați funcția **policy_iteration** pentru calculul politicii optime și a tabelului de utilitate așteptată pentru fiecare stare (celulă din grid) a labirintului.

In [ ]:
def policy_iteration(game: Maze) -> Tuple[Dict[Tuple[int, int], int], Dict[Tuple[int, int], float]]:
    v = {s: 0 for s in game.states}
    policy = {s: choice(game.actions)
              for s in game.states if not game.is_final(s)}


    while True:

        while True:
            delta = 0
            for s in game.states:
                if game.is_final(s):
                    continue
                v_old = v[s]

                v[s] = sum(
                    p * (r + gamma * v[s_next])
                    for (s_next, p, r) in game.effects(s, policy[s])
                )
                delta = max(delta, abs(v[s] - v_old))
            if delta < max_delta:
                break


        policy_stable = True
        for s in game.states:
            if game.is_final(s):
                continue
            old_action = policy[s]

            policy[s] = max(
                game.actions,
                key=lambda a: sum(
                    p * (r + gamma * v[s_next])
                    for (s_next, p, r) in game.effects(s, a)
                )
            )
            if old_action != policy[s]:
                policy_stable = False

        if policy_stable:
            break


    return policy, v

### Cerința 2
Implementați funcția **value_iteration** pentru calculul politicii optime și a tabelului de utilitate așteptată pentru fiecare stare (celulă din grid) a labirintului.

In [ ]:
def value_iteration(game: Maze) -> Tuple[Dict[Tuple[int, int], int], Dict[Tuple[int, int], float]]:
    v = {s: 0 for s in game.states}
    while True:
        delta = 0

        for s in game.states:
            if game.is_final(s):
                continue
            v_old = v[s]

            v[s] = max(
                sum(p * (r + gamma * v[s_next])
                    for (s_next, p, r) in game.effects(s, a))
                for a in game.actions
            )
            delta = max(delta, abs(v[s] - v_old))

        if delta < max_delta:
            break


    policy = {}
    for s in game.states:
        if game.is_final(s):
            continue

        policy[s] = max(
            game.actions,
            key=lambda a: sum(
                p * (r + gamma * v[s_next])
                for (s_next, p, r) in game.effects(s, a)
            )
        )

    return policy, v

## Evaluare

In [ ]:
#@title

for MAP_NAME in ['simple', 'complex', 'be_careful', 'suffer']:
    print("MAP_NAME:", MAP_NAME)

    game = Maze(MAP_NAME)

    print("Policy iteration:")
    policy, v = policy_iteration(game)
    game.print_values(v)
    game.print_policy(policy)

    print("Value iteration:")
    policy, v = value_iteration(game)
    game.print_values(v)
    game.print_policy(policy)

MAP_NAME: simple
Policy iteration:
      |       |       |       |       |       |       |       |      
      |  0.23 |  0.31 |  0.41 |  0.54 |  0.71 |  0.93 |       |      
      |  0.18 |  0.23 |       |       |       |  0.73 |  0.93 |      
      |  0.14 |  0.17 |  0.13 |  0.09 |       |  0.54 |  0.70 |      
      |  0.11 |  0.12 |  0.10 |  0.08 |  0.11 |  0.41 |  0.52 |      
      |       |       |       |       |       |       |       |      
┏━━━━━━━┓
┃→→→→→→B┃
┃↑↑╺━╸↑↑┃
┃↑↑←←A→↑┃
┃↑↑←→↓→↑┃
┗━━━━━━━┛
Value iteration:
      |       |       |       |       |       |       |       |      
      |  0.23 |  0.31 |  0.41 |  0.54 |  0.71 |  0.93 |       |      
      |  0.18 |  0.23 |       |       |       |  0.73 |  0.93 |      
      |  0.14 |  0.17 |  0.13 |  0.09 |       |  0.54 |  0.70 |      
      |  0.11 |  0.12 |  0.10 |  0.08 |  0.11 |  0.41 |  0.52 |      
      |       |       |       |       |       |       |       |      
┏━━━━━━━┓
┃→→→→→→B┃
┃↑↑╺━╸↑↑┃
┃↑↑←←A→↑┃
┃↑↑←→↓→↑